# 19 - ICT Comparaison Villes : Index de Couverture des SVLS en France

> **Dépendance** : ce notebook utilise les données collectées dans **20_GBFS_France_Collecte**  
> `data/gbfs_france/stations_clean.csv` et `output/20_GBFS_France_Collecte/tables_csv/`

---

## Objectif

Calculer et comparer l'**ICT** (Index de Couverture Cyclable/Territoriale) pour l'ensemble des Systèmes de Vélos en Libre-Service (SVLS) à attaches actifs en France, à partir des flux GBFS.

## Définition de l'ICT

L'ICT est un index composite entre 0 et 1 :

$$\text{ICT} = \sqrt{S_{\text{géom}} \times E}$$

| Composante | Formule | Interprétation |
|:-----------|:--------|:---------------|
| **S** | Couverture géométrique (union cercles 300m / hull) | Accessibilité spatiale - quelle fraction de la zone de service est à ≤ 300m d'une station |
| **E** | 1 − Gini(capacités) | Équité - une valeur haute signifie des stations uniformément dimensionnées |
| **ICT** | √(S × E) | Moyenne géométrique - les deux composantes doivent être élevées simultanément |

## Hypothèses
- Seuls les systèmes à **stations d'attaches** sont inclus (filtrage : capacité moyenne ≥ 5 bornes/station)
- La zone de service d'un système est approchée par le **convex hull** de ses stations
- Rayon de marche : **300 m** (standard européen pour l'accès aux transports actifs)


## 1. Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns
from scipy.stats import spearmanr

try:
    from shapely.geometry import Point, MultiPoint
    from shapely.ops import unary_union
    HAS_SHAPELY = True
except ImportError:
    HAS_SHAPELY = False
    print('⚠ shapely non disponible - S_géom calculé par méthode raster numpy uniquement')

ROOT      = Path('../../')
OUT_DIR   = ROOT / 'data' / 'gbfs_france'
NB20_OUT  = ROOT / 'output' / '20_GBFS_France_Collecte' / 'tables_csv'
OUTPUT_NB = ROOT / 'output' / '19_ICT_Comparaison_Villes'

(OUTPUT_NB / 'figures').mkdir(parents=True, exist_ok=True)
(OUTPUT_NB / 'tables_csv').mkdir(exist_ok=True)

plt.style.use('seaborn-v0_8-whitegrid')
WALK_RADIUS_M = 300   # rayon de marche (m)

print('Setup OK')
print(f'Données  → {OUT_DIR}')
print(f'Figures  → {OUTPUT_NB}')

## 2. Chargement & Filtrage des Systèmes à Attaches

In [ ]:
df_stations = pd.read_csv(OUT_DIR / 'stations_clean.csv')
df_profile  = pd.read_csv(NB20_OUT / 'systems_profile.csv')

print(f'Données brutes - {len(df_stations):,} stations | {df_profile["system_id"].nunique()} systèmes')

# Filtrage : systèmes à attaches = capacité moyenne >= 5 et Gini connu
# (exclut les opérateurs free-floating : Dott, Bird, Voi sans bornes)
MEAN_CAP_THRESHOLD = 5

dock_ids = df_profile[
    (df_profile['gini_capacity'].notna()) &
    (df_profile['mean_capacity'] >= MEAN_CAP_THRESHOLD)
]['system_id'].tolist()

df_dock     = df_stations[df_stations['system_id'].isin(dock_ids)].copy()
df_prof_dock = df_profile[df_profile['system_id'].isin(dock_ids)].copy().reset_index(drop=True)

print(f'\nFiltrage (mean_capacity ≥ {MEAN_CAP_THRESHOLD}) :')
print(f'  Systèmes retenus : {len(dock_ids)} / {df_profile["system_id"].nunique()}')
print(f'  Stations totales : {len(df_dock):,}')
print(f'\nTop 10 systèmes par taille :')
print(
    df_prof_dock[['city', 'system_name', 'n_stations', 'mean_capacity', 'E_ict']]
    .sort_values('n_stations', ascending=False)
    .head(10)
    .to_string(index=False)
)

## 3. Composante E - Équité des Capacités

**E = 1 − Gini(capacités)** mesure l'uniformité de la taille des stations.  
E = 1 : toutes les stations ont la même capacité (parfaite équité).  
E = 0 : une seule station concentre toute la capacité (inégalité totale).

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

df_e = df_prof_dock[['city', 'system_name', 'n_stations', 'E_ict', 'gini_capacity']].copy()
df_e = df_e.sort_values('E_ict')

# --- Histogramme E ---
ax = axes[0]
ax.hist(df_e['E_ict'].dropna(), bins=20, color='steelblue', alpha=0.8, edgecolor='white')
ax.axvline(df_e['E_ict'].mean(), color='tomato', lw=2, ls='--',
           label=f'Moyenne = {df_e["E_ict"].mean():.3f}')
ax.axvline(df_e['E_ict'].median(), color='orange', lw=2, ls=':',
           label=f'Médiane = {df_e["E_ict"].median():.3f}')
ax.set_xlabel('E (Équité des capacités)', fontsize=11)
ax.set_ylabel('Nombre de systèmes', fontsize=11)
ax.set_title('Distribution de E sur les SVLS français', fontsize=11, fontweight='bold')
ax.legend(fontsize=10)

# --- Barplot Top/Bottom 20 ---
ax2 = axes[1]
top20 = pd.concat([
    df_e.tail(10),   # meilleurs E
    df_e.head(10),   # pires E
]).drop_duplicates()

colors = ['#2196F3' if e >= df_e['E_ict'].median() else '#F44336'
          for e in top20['E_ict']]
labels = [f"{row['city']} - {row['system_name']}" for _, row in top20.iterrows()]

ax2.barh(range(len(top20)), top20['E_ict'].values, color=colors, alpha=0.85, edgecolor='white')
ax2.set_yticks(range(len(top20)))
ax2.set_yticklabels(labels, fontsize=8)
ax2.axvline(df_e['E_ict'].mean(), color='gray', lw=1.5, ls='--', alpha=0.7)
ax2.set_xlabel('E (Équité)', fontsize=11)
ax2.set_title('Top 10 / Bottom 10 systèmes (équité)', fontsize=11, fontweight='bold')
ax2.set_xlim(0, 1.05)
for i, v in enumerate(top20['E_ict'].values):
    ax2.text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=8)

plt.suptitle(f'Composante E - Équité des Capacités ({len(df_e)} systèmes SVLS France)',
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_NB / 'figures' / '01_composante_E.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'E moyen   : {df_e["E_ict"].mean():.4f}')
print(f'E médian  : {df_e["E_ict"].median():.4f}')
print(f'E std     : {df_e["E_ict"].std():.4f}')
print(f'E min     : {df_e["E_ict"].min():.4f} ({df_e.iloc[0]["city"]})')
print(f'E max     : {df_e["E_ict"].max():.4f} ({df_e.iloc[-1]["city"]})')

## 4. Composante S - Couverture Spatiale Géométrique

**S_géom** mesure quelle fraction de la zone de service est accessible à pied depuis une station.

**Méthode** : approche raster (grille 150 m)
1. Projeter les stations en coordonnées locales (mètres)
2. Pour chaque station, marquer toutes les cellules de grille dans un rayon de 300 m
3. S = cellules couvertes / cellules dans le convex hull des stations

> *Note* : `S = 1` signifie couverture totale de la zone de service.  
> Les systèmes denses (Paris, Nantes) atteignent S ≈ 0.9−1.0 ;  
> les systèmes dispersés (Grand Est, systèmes péri-urbains) ont S < 0.5.

In [ ]:
def compute_coverage_s(stations_df, radius_m=300, grid_m=150):
    """
    Couverture géométrique S d'un système SVLS.
    S = cellules_couvertes (rayon walk_radius) / cellules_hull
    Méthode raster vectorisée numpy - performante pour gros systèmes.
    """
    lats = stations_df['lat'].values
    lons = stations_df['lon'].values

    if len(lats) < 2:
        return 1.0

    lat_c = lats.mean()
    m_per_deg_lat = 111_132.0
    m_per_deg_lon = 111_320.0 * np.cos(np.radians(lat_c))

    # Projection locale (mètres, origine = coin SW)
    x = (lons - lons.min()) * m_per_deg_lon
    y = (lats - lats.min()) * m_per_deg_lat

    # Indices de grille des stations
    ix = np.floor(x / grid_m).astype(int)
    iy = np.floor(y / grid_m).astype(int)

    # Empreinte circulaire (offsets en cellules)
    r_cells = int(np.ceil(radius_m / grid_m))
    off = np.arange(-r_cells, r_cells + 1)
    dxs, dys = np.meshgrid(off, off)
    circle_mask = (dxs**2 + dys**2) <= (radius_m / grid_m)**2
    dxs_c = dxs[circle_mask].ravel()
    dys_c = dys[circle_mask].ravel()

    # Toutes les cellules couvertes (vectorisé)
    all_cx = (ix[:, None] + dxs_c[None, :]).ravel()
    all_cy = (iy[:, None] + dys_c[None, :]).ravel()
    packed  = all_cx.astype(np.int64) * 100_000 + all_cy.astype(np.int64)
    n_covered = len(np.unique(packed))

    # Aire du convex hull (en cellules)
    if HAS_SHAPELY:
        hull = MultiPoint(list(zip(x, y))).convex_hull
        hull_area_m2 = hull.area
    else:
        # Fallback : bbox
        hull_area_m2 = (x.max() - x.min()) * (y.max() - y.min())

    if hull_area_m2 < grid_m**2:
        return 1.0

    hull_cells = hull_area_m2 / grid_m**2
    return float(np.clip(n_covered / hull_cells, 0.0, 1.0))


print(f'Calcul de S_géom pour {len(dock_ids)} systèmes (rayon {WALK_RADIUS_M} m)...')
s_rows = []
for sid in dock_ids:
    grp = df_dock[df_dock['system_id'] == sid]
    s   = compute_coverage_s(grp, radius_m=WALK_RADIUS_M)
    s_rows.append({'system_id': sid, 'S_geom': round(s, 4)})

df_s = pd.DataFrame(s_rows)
print('Calcul terminé.')
print(f'S_géom moyen   : {df_s["S_geom"].mean():.4f}')
print(f'S_géom médian  : {df_s["S_geom"].median():.4f}')
print(f'S_géom std     : {df_s["S_geom"].std():.4f}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

df_s_plot = df_s.merge(df_prof_dock[['system_id', 'city', 'system_name', 'n_stations']])
df_s_plot = df_s_plot.sort_values('S_geom')

# --- Histogramme S ---
ax = axes[0]
ax.hist(df_s_plot['S_geom'], bins=20, color='seagreen', alpha=0.8, edgecolor='white')
ax.axvline(df_s_plot['S_geom'].mean(), color='tomato', lw=2, ls='--',
           label=f'Moyenne = {df_s_plot["S_geom"].mean():.3f}')
ax.set_xlabel('S_géom (couverture spatiale 300 m)', fontsize=11)
ax.set_ylabel('Nombre de systèmes', fontsize=11)
ax.set_title('Distribution de S_géom', fontsize=11, fontweight='bold')
ax.legend(fontsize=10)

# --- Barplot S vs n_stations ---
ax2 = axes[1]
top_s = pd.concat([df_s_plot.tail(12), df_s_plot.head(6)]).drop_duplicates()
colors_s = ['#4CAF50' if s >= 0.5 else '#FF5722' for s in top_s['S_geom']]
labels_s  = [f"{row['city']} ({row['n_stations']} st.)" for _, row in top_s.iterrows()]

ax2.barh(range(len(top_s)), top_s['S_geom'].values, color=colors_s, alpha=0.85, edgecolor='white')
ax2.set_yticks(range(len(top_s)))
ax2.set_yticklabels(labels_s, fontsize=8.5)
ax2.axvline(0.5, color='gray', lw=1, ls='--', alpha=0.7, label='Seuil 50%')
ax2.set_xlabel('S_géom', fontsize=11)
ax2.set_title('Top / Bottom couverture spatiale', fontsize=11, fontweight='bold')
ax2.set_xlim(0, 1.1)
for i, v in enumerate(top_s['S_geom'].values):
    ax2.text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=8)

plt.suptitle('Composante S - Couverture Spatiale Géométrique (rayon 300 m)',
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_NB / 'figures' / '02_composante_S.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Score ICT = √(S × E)

La **moyenne géométrique** √(S × E) pénalise les systèmes où l'une des deux composantes est faible.  
Un système avec S=0.9 et E=0.5 obtient ICT=0.67, inférieur à un système avec S=E=0.75 (ICT=0.75).

In [ ]:
df_ict = (
    df_prof_dock[[
        'system_id', 'city', 'system_name', 'n_stations',
        'mean_capacity', 'total_capacity', 'E_ict',
        'density_st_km2', 'area_bbox_km2'
    ]]
    .merge(df_s, on='system_id')
)

# ICT = √(S × E)
df_ict['ICT'] = np.sqrt(df_ict['S_geom'] * df_ict['E_ict'])
df_ict['ICT'] = df_ict['ICT'].round(4)
df_ict = df_ict.sort_values('ICT', ascending=False).reset_index(drop=True)
df_ict.index += 1  # rang commence à 1

print(f'=== Classement ICT - {len(df_ict)} systèmes SVLS France ===')
print(df_ict[['city', 'system_name', 'n_stations', 'S_geom', 'E_ict', 'ICT']]
      .head(20)
      .to_string())

df_ict.to_csv(OUTPUT_NB / 'tables_csv' / 'ict_scores.csv', index_label='rang')
print('\nICT scores exportés.')

## 6. Classement National des Villes

Visualisation du **classement ICT** avec décomposition S / E pour chaque système.

In [ ]:
# Top 30 systèmes
top_n  = min(30, len(df_ict))
df_top = df_ict.head(top_n).copy()
labels = [f"{row['city']} - {row['system_name']} ({row['n_stations']} st.)"
          for _, row in df_top.iterrows()]

fig, ax = plt.subplots(figsize=(14, top_n * 0.45 + 1.5))

y  = np.arange(top_n)
h  = 0.28

ax.barh(y + h/2, df_top['S_geom'], h, color='#4CAF50', alpha=0.80,
        edgecolor='white', label='S (couverture spatiale)')
ax.barh(y - h/2, df_top['E_ict'],  h, color='#2196F3', alpha=0.80,
        edgecolor='white', label='E (équité capacités)')
ax.scatter(df_top['ICT'], y, marker='D', s=45, color='#FF5722', zorder=5,
           label='ICT = √(S × E)')

# Annotations ICT
for i, v in enumerate(df_top['ICT']):
    ax.text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=8, color='#FF5722', fontweight='bold')

ax.set_yticks(y)
ax.set_yticklabels(labels, fontsize=8.5)
ax.set_xlabel('Score (0 → 1)', fontsize=11)
ax.set_xlim(0, 1.15)
ax.axvline(df_ict['ICT'].mean(), color='gray', lw=1.5, ls='--', alpha=0.7,
           label=f'Moyenne ICT = {df_ict["ICT"].mean():.3f}')
ax.set_title(f'Classement ICT - Top {top_n} Systèmes SVLS France',
             fontsize=12, fontweight='bold')
ax.legend(fontsize=9, loc='lower right')
ax.invert_yaxis()

plt.tight_layout()
plt.savefig(OUTPUT_NB / 'figures' / '03_classement_ICT.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Carte Nationale des Scores ICT

In [ ]:
# Centroïdes des systèmes
centroids = (
    df_dock.groupby('system_id')[['lat', 'lon']]
    .mean()
    .reset_index()
    .rename(columns={'lat': 'lat_c', 'lon': 'lon_c'})
)
df_map = df_ict.reset_index().merge(centroids, on='system_id')

# Garder France métropolitaine
df_metro = df_map[
    (df_map['lat_c'] > 41) & (df_map['lat_c'] < 51.5) &
    (df_map['lon_c'] > -5.5) & (df_map['lon_c'] < 9.5)
]

fig, axes = plt.subplots(1, 2, figsize=(18, 7))
cmap = plt.cm.RdYlGn

# --- Carte ICT ---
ax = axes[0]
sc = ax.scatter(
    df_metro['lon_c'], df_metro['lat_c'],
    c=df_metro['ICT'], cmap=cmap, vmin=0, vmax=1,
    s=df_metro['n_stations'] / 4 + 30,
    alpha=0.85, edgecolors='white', linewidths=0.8, zorder=3
)
plt.colorbar(sc, ax=ax, label='Score ICT', fraction=0.035, pad=0.04)

for _, row in df_metro.iterrows():
    if row['n_stations'] >= 50:
        ax.annotate(
            row['city'], (row['lon_c'], row['lat_c']),
            textcoords='offset points', xytext=(6, 3),
            fontsize=7.5, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.65, edgecolor='none')
        )

ax.set_xlim(-5.5, 9.5)
ax.set_ylim(41, 51.5)
ax.set_xlabel('Longitude', fontsize=10)
ax.set_ylabel('Latitude', fontsize=10)
ax.set_title('Score ICT par Système SVLS\n(taille ∝ nb stations)', fontsize=11, fontweight='bold')

# --- Carte S_géom ---
ax2 = axes[1]
sc2 = ax2.scatter(
    df_metro['lon_c'], df_metro['lat_c'],
    c=df_metro['S_geom'], cmap=cmap, vmin=0, vmax=1,
    s=df_metro['n_stations'] / 4 + 30,
    alpha=0.85, edgecolors='white', linewidths=0.8, zorder=3
)
plt.colorbar(sc2, ax=ax2, label='S_géom', fraction=0.035, pad=0.04)

for _, row in df_metro.iterrows():
    if row['n_stations'] >= 50:
        ax2.annotate(
            f"{row['city']}\nS={row['S_geom']:.2f}",
            (row['lon_c'], row['lat_c']),
            textcoords='offset points', xytext=(6, 3),
            fontsize=7, alpha=0.9,
            bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.65, edgecolor='none')
        )

ax2.set_xlim(-5.5, 9.5)
ax2.set_ylim(41, 51.5)
ax2.set_xlabel('Longitude', fontsize=10)
ax2.set_ylabel('Latitude', fontsize=10)
ax2.set_title('Couverture Spatiale S_géom par Système\n(taille ∝ nb stations)', fontsize=11, fontweight='bold')

plt.suptitle(
    f'Cartographie ICT des SVLS - France ({len(df_metro)} systèmes métropolitains)\n'
    f'Données GBFS - {datetime.now().strftime("%Y-%m-%d")}',
    fontsize=13, fontweight='bold'
)
plt.tight_layout()
plt.savefig(OUTPUT_NB / 'figures' / '04_carte_ICT.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Corrélations ICT × Caractéristiques du Système

In [ ]:
vars_corr = ['n_stations', 'mean_capacity', 'total_capacity',
             'density_st_km2', 'S_geom', 'E_ict', 'ICT']
df_corr = df_ict.reset_index()[vars_corr].dropna()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# --- Scatter S vs E ---
ax = axes[0]
sc = ax.scatter(df_corr['S_geom'], df_corr['E_ict'],
                c=df_corr['ICT'], cmap='RdYlGn', vmin=0, vmax=1,
                s=df_corr['n_stations'] / 5 + 20, alpha=0.8,
                edgecolors='white', linewidths=0.7)
plt.colorbar(sc, ax=ax, label='ICT')
# Isocourbes ICT
s_grid = np.linspace(0.01, 1, 100)
for ict_level in [0.5, 0.65, 0.80, 0.90]:
    e_iso = (ict_level**2) / s_grid
    mask = (e_iso >= 0) & (e_iso <= 1)
    ax.plot(s_grid[mask], e_iso[mask], 'k--', lw=0.8, alpha=0.5)
    ax.text(s_grid[mask][-1], e_iso[mask][-1], f'{ict_level}', fontsize=7.5, alpha=0.6)

# Annoter les systèmes importants
top_names = df_ict.reset_index().nlargest(8, 'n_stations')
for _, row in top_names.iterrows():
    ax.annotate(row['city'], (row['S_geom'], row['E_ict']),
                textcoords='offset points', xytext=(4, 4), fontsize=7.5)

ax.set_xlabel('S_géom (couverture spatiale)', fontsize=11)
ax.set_ylabel('E (équité capacités)', fontsize=11)
ax.set_title('Espace S × E avec isocourbes ICT', fontsize=11, fontweight='bold')
ax.set_xlim(0, 1.05)
ax.set_ylim(0, 1.05)

# --- ICT vs n_stations ---
ax2 = axes[1]
ax2.scatter(df_corr['n_stations'], df_corr['ICT'],
            c=df_corr['density_st_km2'], cmap='YlOrRd',
            s=60, alpha=0.8, edgecolors='white')
r, p = spearmanr(df_corr['n_stations'], df_corr['ICT'])
ax2.set_xlabel('Nombre de stations', fontsize=11)
ax2.set_ylabel('ICT', fontsize=11)
ax2.set_title(f'Taille vs ICT\n(r_Spearman = {r:.3f}, p={p:.3f})', fontsize=11, fontweight='bold')
for _, row in top_names.iterrows():
    ax2.annotate(row['city'], (row['n_stations'], row['ICT']),
                textcoords='offset points', xytext=(4, 4), fontsize=7.5)

# --- ICT vs Densité ---
ax3 = axes[2]
df_plot3 = df_corr[df_corr['density_st_km2'] < 20].copy()  # exclure outliers extrêmes
ax3.scatter(df_plot3['density_st_km2'], df_plot3['ICT'],
            c=df_plot3['n_stations'], cmap='Blues',
            s=60, alpha=0.8, edgecolors='white')
r3, p3 = spearmanr(df_plot3['density_st_km2'], df_plot3['ICT'])
ax3.set_xlabel('Densité (stations/km²)', fontsize=11)
ax3.set_ylabel('ICT', fontsize=11)
ax3.set_title(f'Densité vs ICT\n(r_Spearman = {r3:.3f}, p={p3:.3f})', fontsize=11, fontweight='bold')

plt.suptitle('Corrélations ICT × Caractéristiques Structurelles des Systèmes SVLS',
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_NB / 'figures' / '05_correlations_ICT.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Profils Radar - Top 12 Systèmes

Visualisation des profils multi-axes pour les 12 plus grands systèmes.

In [ ]:
top12 = df_ict.reset_index().nlargest(12, 'n_stations').reset_index(drop=True)

# Normaliser les métriques (min-max) pour le radar
radar_cols = ['n_stations', 'mean_capacity', 'density_st_km2', 'S_geom', 'E_ict', 'ICT']
radar_labels = ['Taille\n(stations)', 'Capacité\nmoyenne', 'Densité\n(st/km²)',
                'S_géom', 'E\n(équité)', 'ICT']

n_axes = len(radar_cols)
angles = np.linspace(0, 2 * np.pi, n_axes, endpoint=False).tolist()
angles += angles[:1]

# Normalisation min-max
normed = top12[radar_cols].copy()
for col in radar_cols:
    col_min, col_max = df_ict.reset_index()[col].min(), df_ict.reset_index()[col].max()
    if col_max > col_min:
        normed[col] = (top12[col] - col_min) / (col_max - col_min)
    else:
        normed[col] = 0.5

palette = plt.cm.tab20.colors

fig, axes = plt.subplots(3, 4, figsize=(20, 14),
                          subplot_kw=dict(polar=True))

for i, (ax, (_, row)) in enumerate(zip(axes.flat, top12.iterrows())):
    values = normed.iloc[i][radar_cols].tolist()
    values += values[:1]

    color = palette[i % len(palette)]
    ax.plot(angles, values, 'o-', linewidth=2, color=color)
    ax.fill(angles, values, alpha=0.25, color=color)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(radar_labels, fontsize=8)
    ax.set_ylim(0, 1)
    ax.set_yticks([0.25, 0.5, 0.75])
    ax.set_yticklabels(['0.25', '0.5', '0.75'], fontsize=6, alpha=0.5)

    ict_val = row['ICT']
    ax.set_title(
        f"{row['city']}\nICT={ict_val:.3f} | {row['n_stations']} st.",
        fontsize=9, fontweight='bold', pad=10
    )

plt.suptitle('Profils Radar - Top 12 Systèmes SVLS par Taille (normalisation min-max)',
             fontsize=13, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig(OUTPUT_NB / 'figures' / '06_radar_top12.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Clustering k-means des Profils de Systèmes

Classification non supervisée des systèmes SVLS selon leurs profils multivariés :
- `S_geom` - couverture spatiale
- `E_ict` - équité des capacités
- `n_stations` (log) - taille du réseau
- `density_st_km2` (log+1) - densité
- `mean_capacity` - taille moyenne des stations

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

feat_cols = ['S_geom', 'E_ict', 'mean_capacity', 'density_st_km2', 'n_stations']
df_feat = df_ict.reset_index()[['system_id', 'city', 'system_name', 'ICT'] + feat_cols].dropna()

# Transformations log pour les variables à distribution longue queue
df_feat = df_feat.copy()
df_feat['log_n_stations']   = np.log1p(df_feat['n_stations'])
df_feat['log_density']      = np.log1p(df_feat['density_st_km2'])

X_raw = df_feat[['S_geom', 'E_ict', 'mean_capacity',
                  'log_density', 'log_n_stations']].values
scaler = StandardScaler()
X      = scaler.fit_transform(X_raw)

# Choix de k par méthode silhouette (k = 2 à 8)
sil_scores = {}
for k in range(2, 9):
    km  = KMeans(n_clusters=k, random_state=42, n_init=10)
    lbl = km.fit_predict(X)
    sil_scores[k] = silhouette_score(X, lbl)

best_k = max(sil_scores, key=sil_scores.get)
print('Scores silhouette :', {k: round(v, 4) for k, v in sil_scores.items()})
print(f'Meilleur k : {best_k} (silhouette = {sil_scores[best_k]:.4f})')

# Appliquer k-means final
km_final = KMeans(n_clusters=best_k, random_state=42, n_init=20)
df_feat['cluster'] = km_final.fit_predict(X)

# PCA 2D pour visualisation
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X)
df_feat['PC1'] = X_pca[:, 0]
df_feat['PC2'] = X_pca[:, 1]

print(f'\nVariance expliquée PCA : PC1={pca.explained_variance_ratio_[0]:.1%}, '
      f'PC2={pca.explained_variance_ratio_[1]:.1%}')
print(f'Total : {pca.explained_variance_ratio_.sum():.1%}')

In [ ]:
# Descriptions des clusters
cluster_stats = (
    df_feat.groupby('cluster')[['S_geom', 'E_ict', 'mean_capacity',
                                 'density_st_km2', 'n_stations', 'ICT']]
    .mean().round(3)
    .sort_values('ICT', ascending=False)
)
cluster_stats['count'] = df_feat.groupby('cluster')['system_id'].count()
print('=== Profil moyen par cluster ===')
print(cluster_stats.to_string())

# Nommer les clusters
cluster_names = {}
for cid, row in cluster_stats.iterrows():
    if row['ICT'] >= 0.80:
        cluster_names[cid] = f'C{cid} - Excellence'
    elif row['ICT'] >= 0.70:
        cluster_names[cid] = f'C{cid} - Performant'
    elif row['ICT'] >= 0.60:
        cluster_names[cid] = f'C{cid} - Intermédiaire'
    else:
        cluster_names[cid] = f'C{cid} - À améliorer'

df_feat['cluster_name'] = df_feat['cluster'].map(cluster_names)

# Sauvegarder
df_feat.to_csv(OUTPUT_NB / 'tables_csv' / 'clusters_profils.csv', index=False)

for cid in sorted(cluster_names):
    cities = df_feat[df_feat['cluster'] == cid]['city'].tolist()
    print(f"\n{cluster_names[cid]} : {cities}")

In [ ]:
palette_k  = plt.cm.Set1.colors

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# --- PCA scatter ---
ax = axes[0]
for cid in sorted(df_feat['cluster'].unique()):
    grp = df_feat[df_feat['cluster'] == cid]
    ax.scatter(grp['PC1'], grp['PC2'], color=palette_k[cid % len(palette_k)],
               s=60, alpha=0.85, edgecolors='white',
               label=cluster_names.get(cid, f'C{cid}'))
    for _, row in grp.nlargest(3, 'n_stations').iterrows():
        ax.annotate(row['city'], (row['PC1'], row['PC2']),
                    textcoords='offset points', xytext=(5, 3), fontsize=7.5)

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})', fontsize=10)
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})', fontsize=10)
ax.set_title('Clustering k-means (PCA 2D)', fontsize=11, fontweight='bold')
ax.legend(fontsize=8, framealpha=0.9)

# --- Silhouette ---
ax2 = axes[1]
ks   = list(sil_scores.keys())
sils = list(sil_scores.values())
ax2.plot(ks, sils, 'o-', color='steelblue', linewidth=2, markersize=8)
ax2.axvline(best_k, color='tomato', ls='--', lw=1.5, label=f'k optimal = {best_k}')
ax2.scatter([best_k], [sil_scores[best_k]], color='tomato', s=120, zorder=5)
ax2.set_xlabel('k (nombre de clusters)', fontsize=11)
ax2.set_ylabel('Silhouette score', fontsize=11)
ax2.set_title('Choix de k - Silhouette', fontsize=11, fontweight='bold')
ax2.legend(fontsize=10)
ax2.set_xticks(ks)

# --- ICT par cluster (boxplot) ---
ax3 = axes[2]
cluster_order = sorted(df_feat['cluster'].unique())
bp_data   = [df_feat[df_feat['cluster'] == c]['ICT'].values for c in cluster_order]
bp_labels = [cluster_names.get(c, f'C{c}') for c in cluster_order]
bp_cols   = [palette_k[c % len(palette_k)] for c in cluster_order]

bp = ax3.boxplot(bp_data, labels=bp_labels, patch_artist=True,
                  medianprops={'color': 'white', 'linewidth': 2})
for patch, color in zip(bp['boxes'], bp_cols):
    patch.set_facecolor(color)
    patch.set_alpha(0.75)

ax3.set_ylabel('ICT', fontsize=11)
ax3.set_title('Distribution ICT par Cluster', fontsize=11, fontweight='bold')
ax3.tick_params(axis='x', labelsize=8)

plt.suptitle(f'Clustering des Profils SVLS - k = {best_k} groupes ({len(df_feat)} systèmes)',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_NB / 'figures' / '07_clustering_kmeans.png', dpi=150, bbox_inches='tight')
plt.show()

## 11. Heatmap des Composantes par Cluster

In [ ]:
# Heatmap : profil moyen de chaque cluster (normalisé)
heat_cols  = ['S_geom', 'E_ict', 'mean_capacity', 'density_st_km2', 'n_stations', 'ICT']
heat_labels = ['S (couverture)', 'E (équité)', 'Cap. moy.', 'Densité', 'Nb stations', 'ICT']

df_heat = (
    df_feat.groupby('cluster_name')[heat_cols]
    .mean()
    .sort_values('ICT', ascending=False)
)
# Normalisation par colonne
df_heat_norm = (df_heat - df_heat.min()) / (df_heat.max() - df_heat.min() + 1e-9)
df_heat_norm.columns = heat_labels

fig, ax = plt.subplots(figsize=(12, max(4, len(df_heat) * 0.9)))
sns.heatmap(
    df_heat_norm,
    annot=df_heat.rename(columns=dict(zip(heat_cols, heat_labels))).round(2),
    fmt='.2f',
    cmap='RdYlGn',
    vmin=0, vmax=1,
    linewidths=0.5,
    annot_kws={'size': 11},
    ax=ax
)
ax.set_title('Profil Moyen par Cluster (couleur = rang normalisé, valeur = brute)',
             fontsize=12, fontweight='bold')
ax.set_ylabel('Cluster', fontsize=11)
ax.tick_params(axis='y', rotation=0)
ax.tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.savefig(OUTPUT_NB / 'figures' / '08_heatmap_clusters.png', dpi=150, bbox_inches='tight')
plt.show()

## 12. Synthèse Finale

In [ ]:
print('=' * 62)
print('  ICT COMPARAISON VILLES - SVLS FRANCE')
print('=' * 62)
print(f'  Date analyse        : {datetime.now().strftime("%Y-%m-%d %H:%M")}')
print(f'  Systèmes analysés   : {len(df_ict)}')
print(f'  (sur {df_profile["system_id"].nunique()} systèmes GBFS France total)')
print()
print(f'  ICT moyen           : {df_ict["ICT"].mean():.4f}')
print(f'  ICT médian          : {df_ict["ICT"].median():.4f}')
print(f'  ICT min             : {df_ict["ICT"].min():.4f}')
print(f'  ICT max             : {df_ict["ICT"].max():.4f}')
print()
print('  Top 5 systèmes :')
for _, row in df_ict.reset_index().head(5).iterrows():
    print(f'    {row["city"]:25s} ICT={row["ICT"]:.3f}  '
          f'(S={row["S_geom"]:.3f}, E={row["E_ict"]:.3f})')
print()
print('  Bottom 5 systèmes :')
for _, row in df_ict.reset_index().tail(5).iterrows():
    print(f'    {row["city"]:25s} ICT={row["ICT"]:.3f}  '
          f'(S={row["S_geom"]:.3f}, E={row["E_ict"]:.3f})')
print()
print(f'  Clusters k-means    : {best_k} groupes')
for name, cnt in df_feat['cluster_name'].value_counts().items():
    print(f'    {name:35s}: {cnt} systèmes')
print()
print('  FICHIERS EXPORTÉS :')
for f in sorted((OUTPUT_NB / 'tables_csv').glob('*.csv')):
    print(f'    {f.name}')
print()
print('  PROCHAINES ÉTAPES :')
print('  • Nb 21 - Voronoï national : zones blanches sans SVLS')
print('  • Nb 22 - Modèle de diffusion spatiale (quelles villes manquent de SVLS ?)')
print('=' * 62)